In [ ]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

In [40]:
import requests
import dlt

qa_dataset = requests.get("https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1").json()

@dlt.resource
def qa_documents():
  for course in qa_dataset:
    yield course["documents"]

pipeline = dlt.pipeline(pipeline_name="from_json", destination="lancedb", dataset_name="qanda")

load_info = pipeline.run(qa_documents, table_name="documents")

print(load_info)

Pipeline from_json load step completed in 0.18 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7de7292497e0> location to store data
Load package 1721624119.67608 is LOADED and contains no failed jobs


In [ ]:
import lancedb

db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents']


In [ ]:
db_table = db.open_table("qanda___documents")

db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,52bc2201-55fc-5852-a828-a86ece3e7ebd,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721619975.5614681,ZP28S0+tW9KizQ
1,10ca65f8-6d01-55ff-b17f-78eed0d9c70a,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721619975.5614681,mdXFu1R2GV73kw
2,50cc310a-57f6-5147-8c3e-1abec5962baf,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721619975.5614681,1WeQd5vKiNsHWA
3,459a8529-b647-5dc6-bca2-941229c0a4c1,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721619975.5614681,4bO/OQ6oxbVGHA
4,a8f5a213-d0d3-5046-a6e2-d3b0b5df62bc,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721619975.5614681,sYHa19pGKRdFpA
...,...,...,...,...,...,...
943,05d56fe4-77ae-548e-a7dd-7cea1bd2a041,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721619975.5614681,8CfTwDYapfj2QA
944,49795fa0-7170-55ee-946f-77c97a6982f0,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721619975.5614681,N43Fync716Q1jw
945,2e6872f0-2de1-5e16-8593-8461dafc80e3,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721619975.5614681,qxoaTHtWgnVsrA
946,55379761-24b7-5b3f-9fa8-272bb5999ea8,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721619975.5614681,f1kFF9VbOH31FQ


In [ ]:
import os
from dlt.destinations.adapters import lancedb_adapter

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

pipeline = dlt.pipeline(pipeline_name="from_json_embedded", destination="lancedb", dataset_name="qanda_embedded")

load_info = pipeline.run(lancedb_adapter(qa_documents, embed=["text", "question"]), table_name="documents")
print(load_info)

_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': False}, {'name': 'schema', 'data_type': 'text', 'nullable': False}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
documents
[{'name': 'text', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': '_d

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
UPLOAD
Pipeline from_json_embedded load step completed in 24.73 seconds
1 load package(s) were loaded to destination LanceDB and into dataset qanda_embedded
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7de71bb0ea70> location to store data
Load package 1721620317.032623 is LOADED and contains no failed jobs


In [ ]:
db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [ ]:
db_table = db.open_table("qanda_embedded___documents")

db_table.to_pandas()

,id__,vector__,text,section,question,_dlt_load_id,_dlt_id
0,b4db9e93-b2fd-5366-99d7-3657862f266c,"[-0.00035095983, -0.062014256, -0.03799991, 0....",The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1721620317.032623,hjcDFwAJF4jHSg
1,01229bde-7934-570f-a59d-3c00fe21e02a,"[0.020011423, -0.011535534, 0.0130172055, -0.0...",GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1721620317.032623,lyvr71qwcrDgkw
2,32556468-faec-5c06-9c27-7e5addab8c22,"[0.014857549, -0.06664994, -0.013571203, 0.023...","Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1721620317.032623,tmLwwYSXs9ZyiQ
3,f9a14da9-ec6d-5055-bfd5-0457a2af9d68,"[-0.023312101, -0.09461491, 0.056361604, -0.00...",You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1721620317.032623,bJIx1A6Ovj6+nQ
4,303fa9a2-6a8f-5d1e-afba-7fec9578da1e,"[0.026537659, -0.017796658, 0.0021156375, 0.00...",You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1721620317.032623,LswAaJDUSDuC+g
...,...,...,...,...,...,...,...
943,f2707461-6359-5737-bdd5-e2aaa2f8b777,"[0.016619304, -0.033603117, -0.09334721, -0.02...",Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1721620317.032623,B/LlwIWCgTFVsQ
944,e496fce0-27c8-5094-b2ea-d377ac607e8e,"[0.026872871, -0.0019949335, 0.008369085, -0.0...",Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1721620317.032623,bHm08kpPzIzBdg
945,7ef49b9a-09e8-5e87-b669-d42df925095d,"[0.03513752, 0.056265578, 0.024428517, -0.0651...",Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1721620317.032623,affloh36MVAhKg
946,2e2d1552-0ef9-56d2-9917-45e23a7a1bb6,"[0.03380982, -0.0031219546, 0.0017484119, 0.01...",Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1721620317.032623,KSy1EBK+rGsUhw


In [ ]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

In [ ]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

In [ ]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="homework")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

int64
version
int64
engine_version
timestamp[us, tz=UTC]
inserted_at
string
schema_name
string
version_hash
string
schema
_dlt_version
[]
string
load_id
string
schema_name
int64
status
timestamp[us, tz=UTC]
inserted_at
string
schema_version_hash
_dlt_loads
[]
int64
version
int64
engine_version
string
pipeline_name
string
state
timestamp[us, tz=UTC]
created_at
string
version_hash
string
_dlt_load_id
string
_dlt_id
_dlt_pipeline_state
[]
homework
[{'name': 'block_id', 'nullable': False, 'primary_key': True, 'data_type': 'text'}, {'name': 'block_type', 'data_type': 'text', 'nullable': True}, {'name': 'content', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'dedup_sort': 'desc', 'name': 'last_edited_time', 'data_type': 'timestamp', 'nullable': True}, {'name': 'inserted_at_time', 'data_type': 'timestamp', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
UPLOAD

In [ ]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,71e89a85-ae0b-5b68-866b-bd3922ec7548,"[-0.05858811, -0.07540446, 0.033775203, 0.0096...",c0262981-b5f1-4a57-a91f-2e75f649b86c,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-18 14:00:00+00:00,2024-07-22 04:20:42.750016+00:00,1721622041.764276,5Kjm3lHJMtFQlw
1,a28e913f-761f-5684-8cd5-0d0c49e0338c,"[-0.004968941, -0.003911972, 0.028705625, 0.00...",faacf4ec-90be-4e96-b8b9-29b5112bc7ca,paragraph,Employees receive [20 days] of Paid Time Off (...,2024-06-26 09:03:00+00:00,2024-07-22 04:20:42.752135+00:00,1721622041.764276,1ED/Gj4JvbeIhg
2,a18932d9-1583-5c42-bd0d-0f96738c5e6c,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",e6021a51-f403-4950-80c2-ebff005c7289,paragraph,Our company observes the following holidays: N...,2024-06-26 09:08:00+00:00,2024-07-22 04:20:42.752346+00:00,1721622041.764276,zH28iajWsX8S9g
3,93661874-13a2-5a43-bed8-868005dfd5e2,"[-0.0131553095, 0.008382407, 0.017044391, 0.05...",b8f4cc6d-c28c-4071-9545-caadce5eb37b,paragraph,These holidays are considered “off-days” for m...,2024-06-26 09:09:00+00:00,2024-07-22 04:20:42.752507+00:00,1721622041.764276,46cgy9TZLl+X7w
4,b220778f-1118-5c22-b614-3bc0fd0a602b,"[0.027987516, 0.067343615, 0.03980646, 0.00774...",ea7a1beb-6874-4f41-966d-dc1f80a1f635,paragraph,Employees who are unable to work due to illnes...,2024-06-26 09:11:00+00:00,2024-07-22 04:20:42.752674+00:00,1721622041.764276,F+/SWY29/B3G9g
5,d0f801ba-d3cc-5252-ad6e-3285662b609c,"[0.03252615, 0.008159476, 0.08443566, 0.055641...",bd7a9110-fac5-4270-9493-4039ca67b467,paragraph,Losing a loved one is traumatizing. If this ha...,2024-06-26 09:17:00+00:00,2024-07-22 04:20:42.752831+00:00,1721622041.764276,cq9X5BG1RL84ig
6,579b97f9-a5e2-53af-b4f7-efc9ad5105ad,"[-0.007314052, 0.014710642, -0.019091193, 0.02...",b1718dee-8c0f-4189-8c75-0e8c7844a501,paragraph,"In accordance with German law, we offer a comp...",2024-06-26 09:20:00+00:00,2024-07-22 04:20:42.752999+00:00,1721622041.764276,hBJu3WaJVys4Pg
7,a9083b7e-22cc-5b1f-8040-cb7aa1f72338,"[-0.031538427, 0.0342599, -0.027282646, 0.0275...",5bfa90c5-461d-406a-9324-a1dd54bad0d5,paragraph,We recognize the vital role that fathers and p...,2024-06-26 09:21:00+00:00,2024-07-22 04:20:42.753152+00:00,1721622041.764276,gIJTv/FsELqPEA
8,6adeb540-d180-5d40-bc84-c40e5c173ea1,"[-0.03892389, 0.1208173, 0.046208583, -0.00543...",baac0ba4-9b60-450e-8cc1-1e6e2a0fb7d9,paragraph,"In this section, we describe what we offer to ...",2024-07-03 17:34:00+00:00,2024-07-22 04:20:52.316316+00:00,1721622041.764276,XIBzTNYu0J64+g
9,cffdb1bb-a146-5e90-8fbb-a1d577a2a98e,"[-0.07571499, 0.14543605, 0.0011521844, -0.024...",0e429073-6383-4918-8961-fcc66346067f,paragraph,{edited} Employee health is important to us. W...,2024-07-18 17:28:00+00:00,2024-07-22 04:20:52.316624+00:00,1721622041.764276,lY8AQ4OSuIvp8w


 ---

In [ ]:
db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,6adeb540-d180-5d40-bc84-c40e5c173ea1,"[-0.03892389, 0.1208173, 0.046208583, -0.00543...",baac0ba4-9b60-450e-8cc1-1e6e2a0fb7d9,paragraph,"In this section, we describe what we offer to ...",2024-07-03 17:34:00+00:00,2024-07-08 15:30:04.270715+00:00,1720452602.3108296,+LXDpddrXOJUXg
1,cffdb1bb-a146-5e90-8fbb-a1d577a2a98e,"[-0.0799329, 0.13477285, 0.0053403154, -0.0298...",0e429073-6383-4918-8961-fcc66346067f,paragraph,Employee health is important to us. We don’t d...,2024-06-26 08:46:00+00:00,2024-07-08 15:30:04.272891+00:00,1720452602.3108296,gDimzresa+mpsg
2,25cd721d-fd64-517f-9b3b-34e3fad3522e,"[-0.109743185, 0.10586075, 0.003290699, -0.021...",f4e006d7-9b38-49e9-94cf-552beaa75773,paragraph,Our company is dedicated to maintaining a safe...,2024-07-03 17:26:00+00:00,2024-07-08 15:30:04.273102+00:00,1720452602.3108296,1OZTtNPR9Ab8uA
3,c75b7ef9-96b6-551b-9cdd-795bbe01bb6e,"[0.050755523, -0.06461991, 0.06527383, 0.01465...",71618ca5-6c62-4b66-bc0f-3d855e0c4b8b,paragraph,If your job doesn’t require you to be present ...,2024-06-26 08:52:00+00:00,2024-07-08 15:30:04.273273+00:00,1720452602.3108296,t3k2vgTDh3Fc7Q
4,7a69c4c0-cd55-5090-903e-facf23eadde5,"[0.00052337867, -0.054883413, 0.043573413, -0....",cd15aaf5-6cdc-4a13-835c-2181fd7bf81e,paragraph,Remote working refers to working from a non-of...,2024-07-03 17:19:00+00:00,2024-07-08 15:30:04.273443+00:00,1720452602.3108296,R5wmdAkkOOmdpg
5,ff1141dc-88f6-500a-a8c3-c18e37661650,"[0.03802633, -0.021509705, 0.04752782, 0.06470...",a4b2f0c9-e0c8-4b3c-81e7-ef624809977d,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:32:00+00:00,2024-07-08 15:30:04.273595+00:00,1720452602.3108296,qxJLuSZQaZq/fw
6,a28e913f-761f-5684-8cd5-0d0c49e0338c,"[-0.004968941, -0.003911972, 0.028705625, 0.00...",faacf4ec-90be-4e96-b8b9-29b5112bc7ca,paragraph,Employees receive [20 days] of Paid Time Off (...,2024-06-26 09:03:00+00:00,2024-07-08 15:30:04.662656+00:00,1720452602.3108296,/oDmr/7ulovYhQ
7,a18932d9-1583-5c42-bd0d-0f96738c5e6c,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",e6021a51-f403-4950-80c2-ebff005c7289,paragraph,Our company observes the following holidays: N...,2024-06-26 09:08:00+00:00,2024-07-08 15:30:04.662820+00:00,1720452602.3108296,9I7CX2AaReDvng
8,93661874-13a2-5a43-bed8-868005dfd5e2,"[-0.0131553095, 0.008382407, 0.017044391, 0.05...",b8f4cc6d-c28c-4071-9545-caadce5eb37b,paragraph,These holidays are considered “off-days” for m...,2024-06-26 09:09:00+00:00,2024-07-08 15:30:04.662974+00:00,1720452602.3108296,HiI2XYEzmAEQMA
9,b220778f-1118-5c22-b614-3bc0fd0a602b,"[0.027987516, 0.067343615, 0.03980646, 0.00774...",ea7a1beb-6874-4f41-966d-dc1f80a1f635,paragraph,Employees who are unable to work due to illnes...,2024-06-26 09:11:00+00:00,2024-07-08 15:30:04.663125+00:00,1720452602.3108296,UwkDg5Htn2kTvA


In [ ]:
print(db.table_names())

['notion_pages____dlt_loads', 'notion_pages____dlt_pipeline_state', 'notion_pages____dlt_version', 'notion_pages___dltSentinelTable', 'notion_pages___employee_handbook', 'notion_pages___homework', 'qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents', 'qanda_embedded____dlt_loads', 'qanda_embedded____dlt_pipeline_state', 'qanda_embedded____dlt_version', 'qanda_embedded___dltSentinelTable', 'qanda_embedded___documents']


In [ ]:
dbtable = db.open_table("notion_pages___homework")

table_df = dbtable.to_pandas()

In [39]:
table_df.shape

(17, 9)

In [30]:
table_df['last_edited_time'].max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


2. Start Ollama using `ollama serve`. This needs to run in the backgound - so we run it using `nohup` (to see the output log, open nohup.out).

In [35]:
!nohup ollama serve > nohup.out 2>&1 &

In [36]:
%%capture
!ollama pull llama2-uncensored

In [ ]:
!pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 770.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [ ]:
import ollama

In [ ]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [33]:
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___employee_handbook")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [37]:
main()

You: PTO days
Assistant: Thank you for asking your question! Based on the paragraph provided by the user, here is my response: "We encourage you to take your Paid Time Off (PTO) throughout the year. However, if you have any questions or concerns about your PTO policy, please contact HR and they will be happy to assist you. The 20 days of PTO that you receive per year is a great benefit and we encourage you to use it wisely. Additionally, when scheduling time off for holidays, if you are a non-exempt employee, you will receive your regular hourly rate with a premium for working on a holiday. If you are an exempt employee, you will receive an additional day of PTO that must be taken within 12 months after the holiday."


KeyboardInterrupt: Interrupted by user